### IMU preintegration from scratch and comparison against Pypose IMU preintegrator on KITTI dataset (See [link](https://pypose.org/tutorials/imu/imu_integrator_tutorial))

Author: Jongwon Lee
Date: April 23rd, 2024


### IMU preintegration from scratch (for Symforce usability)

Do all imports

In [1]:
# For numerical methods
import numpy as np
from scipy.linalg import expm, logm
from scipy.spatial.transform import Rotation as R

# For image processing and visualization of results
import matplotlib.pyplot as plt

# For optimization with symforce
import symforce
symforce.set_epsilon_to_symbol()
import symforce.symbolic as sf
from symforce.values import Values
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
import sym

Define `Preintegrator` class and utility functions

In [2]:
def Hat(w):
    '''
    https://ingmec.ual.es/~jlblanco/papers/jlblanco2010geometry3D_techrep.pdf
    '''
    assert(len(w) == 3)

    x, y, z = w

    Hat = np.array([[0, -z, y],
                    [z, 0, -x],
                    [-y, x, 0]])

    return Hat

def Vee(R):
    '''
    https://ingmec.ual.es/~jlblanco/papers/jlblanco2010geometry3D_techrep.pdf
    '''
    assert(R.shape == (3,3))

    Vee = np.array([R[2,1], -R[2,0], R[1,0]])

    return Vee

def J_SO3(phi):
    '''
    Refer to Eq. 8 in Forster
    '''
    assert(len(phi) == 3)

    phi_norm = np.linalg.norm(phi)

    return np.eye(3) \
    - (1 - np.cos(phi_norm)) / phi_norm ** 2 * Hat(phi) \
    + (phi_norm - np.sin(phi_norm)) / phi_norm ** 3 * Hat(phi) * Hat(phi)


class Preintegrator:
    def __init__(self,
                 dt,
                 acc_cov, gyr_cov,
                 acc_bias=np.zeros(3), gyr_bias=np.zeros(3),
                 ):
        self.dt = dt

        self.Sigma_eta = np.block([[gyr_cov * np.eye(3),        np.zeros((3,3))],
                                   [    np.zeros((3,3)),    acc_cov * np.eye(3)]]) / self.dt
        self.Sigma = None

        self.acc_bias = acc_bias
        self.gyr_bias = gyr_bias

    def set_bias(self, acc_bias, gyr_bias):
        self.acc_bias = acc_bias
        self.gyr_bias = gyr_bias

    def set_cov(self, Cov):
        self.Sigma = Cov

    def integrate(self, acc_meas, gyr_meas):
        assert(len(acc_meas) == len(gyr_meas))

        Sigma = np.zeros((9,9)) if self.Sigma is None else self.Sigma
        dR = np.eye(3)
        dv = np.zeros(3)
        dp = np.zeros(3)

        for acc, gyr in zip(acc_meas, gyr_meas):
            Sigma_i_k = Sigma.copy()
            dR_i_k = dR.copy()
            dv_i_k = dv.copy()
            dp_i_k = dp.copy()

            dR_k_k1 = expm(Hat((gyr - self.gyr_bias) * self.dt))

            # Propagate covariance
            # Refer to Eq.A.9 in Forster supplementary
            A = np.block([[dR_k_k1.T, np.zeros((3,3)), np.zeros((3,3))],
                          [- dR_i_k @ Hat(acc - self.acc_bias) * self.dt, np.eye(3), np.zeros((3,3))],
                          [- 0.5 * dR_i_k @ Hat(acc - self.acc_bias) * self.dt ** 2, np.eye(3) * self.dt, np.eye(3)],
                          ])
            B = np.block([[J_SO3((gyr - self.gyr_bias) * self.dt), np.zeros((3,3))],
                          [np.zeros((3,3)), dR_i_k * self.dt],
                          [np.zeros((3,3)), 0.5 * dR_i_k * self.dt ** 2]])

            Sigma = A @ Sigma_i_k @ A.T + B @ self.Sigma_eta @ B.T

            # Propagate increment of pose and velocity
            # Refer to Eq.A.10 in Forster supplementary
            dR = dR_i_k @ dR_k_k1
            dv = dv_i_k + dR_i_k @ (acc - self.acc_bias) * self.dt
            dp = dp_i_k + dv_i_k * self.dt + 0.5 * dR_i_k @ (acc - self.acc_bias) * self.dt ** 2

        return dR, dv, dp, Sigma


An example of how to use `Preintegrator` class

In [3]:
integrator = Preintegrator(0.01, np.eye(3), np.eye(3))

acc_meas = np.zeros((10,3))
gyr_meas = np.random.random((10,3)) / 1e3

dR, dv, dp, Sigma = integrator.integrate(acc_meas, gyr_meas)

Define rediuals for Symforce use

In [4]:
# See Eq. 37 in Forster or Eq.A.21 in Supplementary
def imu_preintegration_residual(
        # Keys subject to optimize
        pose_i: sf.Pose3,
        vel_i: sf.V3,
        pose_j: sf.Pose3,
        vel_j: sf.V3,
        acc_bias: sf.V3,
        gyr_bias: sf.V3,
        # IMU preintegrator
        integrator: Preintegrator,
        # IMU measuremets
        acc_meas: np.ndarray,
        gyr_meas: np.ndarray,
        epsilon: sf.Scalar,
        ) -> sf.V9:
    assert(len(acc_meas) == len(gyr_meas))

    # Perform IMU preintegration
    integrator.set_bias(acc_bias, gyr_bias)
    dR, dv, dp, Sigma = integrator.integrate(acc_meas, gyr_meas)

    gravity = np.array([0., 0., -9.81])
    dt_ij = integrator.dt * len(acc_meas)

    T_i = pose_i.to_homogenous_matrix()
    T_j = pose_j.to_homogenous_matrix()

    R_i = T_i[:3,:3]
    p_i = T_i[:3, -1]
    R_j = T_j[:3,:3]
    p_j = T_j[:3, -1]

    # Refer to Eq. 37 in Forster or Eq.A.21 in Supplementary
    residual_dR = Vee(logm(dR.T @ R_i.T @ R_j))
    residual_dv = R_i.T @ (vel_j - vel_i - gravity * dt_ij) - dv
    residual_dp = R_i.T @ (p_j - p_i - vel_i * dt_ij - 0.5 * gravity * dt_ij ** 2) - dp

    # Normalize residuals by the squareroot inverse of covariance
    # FIXME: Aren't there better ways to perform this?
    sqrt_info = np.sqrt(np.linalg.inv(Sigma))

    return sqrt_info * sf.V9(residual_dR.col_join(residual_dv).col_join(residual_dp))

# See Eq. 40 in Forster
def imu_bias_residual(
        # Keys subject to optimize
        acc_bias_i: sf.V3,
        gyr_bias_i: sf.V3,
        acc_bias_j: sf.V3,
        gyr_bias_j: sf.V3,
        acc_cov: sf.Scalar,
        gyr_cov: sf.Scalar,
        epsilon: sf.Scalar,
        ) -> sf.V6:
    # Compute the square root of the inverse covariance directly
    gyr_info = 1 / sf.sqrt(gyr_cov)
    acc_info = 1 / sf.sqrt(acc_cov)

    # Create a diagonal matrix with these values
    sqrt_info = sf.Matrix.diag(sf.Matrix([gyr_info, gyr_info, gyr_info, acc_info, acc_info, acc_info]))

    return sqrt_info * sf.V6((gyr_bias_j - gyr_bias_i).col_join(acc_bias_j - acc_bias_i))

Symforce optimization example

In [5]:
sigma_acc_wn = 1e-4  # accelerometer white noise sigma
sigma_gyr_wn = 1e-6  # gyroscope white noise sigma
sigma_acc_rw = 1e-5  # accelerometer random walk sigma
sigma_gyr_rw = 1e-7  # gyroscope random walk sigma

dt = 0.1

Cov_acc_bias = sigma_acc_rw ** 2  * dt
Cov_gyr_bias = sigma_gyr_rw ** 2  * dt

integrator = Preintegrator(dt, sigma_acc_wn**2, sigma_gyr_wn**2)

# Generate simulated imu data
acc_meas = np.zeros((10,3))
gyr_meas = np.random.random((10,3)) / 1e3

# Define initial values
initial_values = Values(
    ### Optimized variables
    # Variables at timestep i
    pose_i=sym.Pose3(R=sym.Rot3.from_rotation_matrix(np.eye(3)), t=np.zeros(3)),
    vel_i=np.zeros(3),
    acc_bias_i=np.zeros(3),
    gyr_bias_i=np.zeros(3),
    # Variables at timestep j
    pose_j=sym.Pose3(R=sym.Rot3.from_rotation_matrix(np.eye(3)), t=np.zeros(3)),
    vel_j=np.zeros(3),
    acc_bias_j=np.zeros(3),
    gyr_bias_j=np.zeros(3),
    ## Constant variables / classes
    # IMU preintegrator, measurements, and metadata
    integrator=integrator,
    acc_meas=acc_meas,
    gyr_meas=gyr_meas,
    Cov_acc_bias=Cov_acc_bias,
    Cov_gyr_bias=Cov_gyr_bias,
    dt_ij=dt,
    # Epsilon
    epsilon=sym.epsilon,
)

# Define optimized keys
optimized_keys = ['pose_i', 'vel_i', 'acc_bias_i', 'gyr_bias_i', 
                  'pose_j', 'vel_j', 'acc_bias_j', 'gyr_bias_j']

# Add factors
factors = []

factors.append(Factor(
                residual=imu_preintegration_residual,
                keys=[
                    'pose_i', 'vel_i',
                    'pose_j', 'vel_j',
                    'acc_bias_i',
                    'gyr_bias_i',
                    'integrator',
                    'acc_meas',
                    'gyr_meas',
                    'epsilon',
                ],
            ))
factors.append(Factor(
                residual=imu_bias_residual,
                keys=[
                    'acc_bias_i',
                    'gyr_bias_i',
                    'acc_bias_j',
                    'gyr_bias_j',
                    'Cov_acc_bias',
                    'Cov_gyr_bias',
                    'epsilon',
                ],
))

# Solve optimization
optimizer = Optimizer(
    factors=factors,
    optimized_keys=optimized_keys,
    debug_stats=True,
    params=Optimizer.Params(
        iterations=100,
        use_diagonal_damping=True,
        lambda_down_factor=0.1,
        lambda_up_factor=5.,
        early_exit_min_reduction=1e-4,
    ),
)

result = optimizer.optimize(initial_values)

NotImplementedError: <class '__main__.Preintegrator'> is not registered under StorageOps